#0) Imports and Installs

In [ ]:
# install the main library YFinance

!pip install yfinance

In [ ]:
# IMPORTS

import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# Limit window size and enable scrolling of outputs
from IPython.display import Javascript

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 75})'''))

<IPython.core.display.Javascript object>

# Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list (stockanalysis.com/ipos/withdrawn), collect and process the data to find out which company type saw the most withdrawn IPO value.

**Steps:**

1. Use pandas.read_html() with the URL above to load the IPO withdrawal table into a DataFrame. It is a similar process to Code Snippet 1 discussed at the livestream. You should get 99 entries.
2. Create a new column called Company Class, categorizing company names based on patterns like:
* “Acquisition Corp” or “Acquisition Corporation” → Acq.Corp
* “Inc” or “Incorporated” → Inc
* “Group” → Group
* “Holdings” → Holdings
* “Ltd” or “Limited” → Ltd
* Others → Other <br>
Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
3. Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column). Examples:
'$8.00-$10.00' → 9.0
'$5.00' → 5.0
'-' → None
4. Convert Shares Offered to numeric, clean missing or invalid values.
5. Create a new column:
Withdrawn Value = Shares Offered * Avg Price (71 non-null values)
6. Group by Company Class and calculate total withdrawn value.
7. Answer: Which class had the highest total value of withdrawals?


#### Step 1: Use pandas.read_html() with the URL above to load the IPO withdrawal table in

In [ ]:
# Step 1: Use pandas.read_html() with the URL above to load the IPO withdrawal table into a DataFrame.

from io import StringIO

def get_ipos_withdrawn():
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:
# Create new dataframe

ipos_withdrawn = get_ipos_withdrawn()
ipos_withdrawn.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [ ]:
# Show first and last 5 entries in dataframe
ipos_withdrawn

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


In [ ]:
# Look at unique values of 'Company Name' before cleaning -  only 100 entries

ipos_withdrawn['Company Name'].unique()

array(['Odyssey Therapeutics, Inc.', 'Unifoil Holdings, Inc.',
       'Aurion Biotech, Inc.', 'PHI Group, Inc.', 'One Power Company',
       'The Great Restaurant Development Holdings Limited',
       'Caring Brands, Inc.', 'Sequoia Vaccines, Inc.',
       'Shenni Holdings Limited', 'Key Mining Corp.',
       'GenEmbryomics Limited', 'Job Aire Group Inc.', 'Pyro AI Inc.',
       'Oranco, Inc.', 'Brilliance Group', 'FD Technology Inc.',
       'NYIAX, Inc.', 'Turo Inc.', 'HTL Capital Ltd.',
       'Clarios International, Inc.', 'Metros Development Co., Ltd.',
       'Navios South American Logistics, Inc.', 'AgiiPlus Inc.',
       'Republic Power Group Limited', 'APRINOIA Therapeutics Inc.',
       'Breathe BioMedical Inc.', 'Novelis Inc.',
       'Prospect Energy Holdings Corp.', 'First Person Ltd.',
       'Droneify Holdings Limited', 'EMulate Therapeutics, Inc.',
       'LeeWay Services, Inc.', 'Wytec International, Inc.',
       'QinHong International Group', 'F3 Platform Biologics, 

#### Step 2: Create a new column called Company Class, categorizing company names based on patterns.

In [ ]:
# Step 2: Create a new column called Company Class, categorizing company names based on patterns like:

def clean_company_name(name):
    """
    Cleans the company name by removing periods and commas.
    """
    if pd.isna(name):
        return name
    return str(name).replace('.', '').replace(',', '').strip()

def categorize_company(name):
    """
    Categorizes company names based on predefined patterns,
    after cleaning by removing periods and commas.
    """
    cleaned_name = clean_company_name(name)

    if pd.isna(cleaned_name):
        return 'Other'

    name_lower = cleaned_name.lower()
    words = name_lower.split()

    if "acquisition corp" in name_lower or "acquisition corporation" in name_lower:
        return "Acq.Corp"
    elif "inc" in words or "incorporated" in words:
        return "Inc"
    elif "group" in words:
        return "Group"
    elif "holdings" in words:
        return "Holdings"
    elif "ltd" in words or "limited" in words:
        return "Ltd"
    else:
        return "Other"

In [ ]:
# Clean company names before categorization
ipos_withdrawn['Clean Company Name'] = ipos_withdrawn['Company Name'].apply(clean_company_name)
ipos_withdrawn['Clean Company Name']

,Clean Company Name
0,Odyssey Therapeutics Inc
1,Unifoil Holdings Inc
2,Aurion Biotech Inc
3,PHI Group Inc
4,One Power Company
...,...
95,Freehold Properties Inc
96,Chobani Inc
97,iFIT Health & Fitness Inc
98,Gerson Lehrman Group Inc


In [ ]:
# Create column with categorizations
ipos_withdrawn['Category'] = ipos_withdrawn['Clean Company Name'].apply(categorize_company)
ipos_withdrawn['Category']

,Category
0,Inc
1,Inc
2,Inc
3,Inc
4,Other
...,...
95,Inc
96,Inc
97,Inc
98,Inc


In [ ]:
ipos_withdrawn['Category'].value_counts()

,count
Category,
Inc,51
Acq.Corp,21
Ltd,12
Other,6
Holdings,6
Group,4


In [ ]:
# Show the companies in the 'Other' category
ipos_withdrawn[ipos_withdrawn['Category'] == 'Other']

,Symbol,Company Name,Price Range,Shares Offered,Clean Company Name,Category
4,ONE,One Power Company,-,-,One Power Company,Other
9,KMCM,Key Mining Corp.,$2.25,4444444,Key Mining Corp,Other
53,CLLB,"CoLabs Intâl, Corp.",$4.50,1300000,CoLabs Intâl Corp,Other
74,TSIV,Twelve Seas Investment Company IV TMT,$10.00,20000000,Twelve Seas Investment Company IV TMT,Other
86,FSPR,Four Springs Capital Trust,$13.00 - $15.00,18000000,Four Springs Capital Trust,Other
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000,hearcom NV,Other


#### Step 3: Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column).

In [ ]:
# Step 3 Define a new field Avg. price by parsing the Price Range field (create a function and apply it to the Price Range column).

def clean_price_range(price_range):
    """
    Cleans the price range string by converting to string, stripping whitespace, and removing '$'.
    """
    if pd.isna(price_range): # Handle NaN before string operations
        return None
    return str(price_range).strip().replace('$', '')


ipos_withdrawn['Clean Price Range'] = ipos_withdrawn['Price Range'].apply(clean_price_range)
ipos_withdrawn['Clean Price Range']

,Clean Price Range
0,-
1,3.00 - 4.00
2,-
3,-
4,-
...,...
95,-
96,-
97,18.00 - 21.00
98,-


In [ ]:
def avg_price_range(price_range):
    """
    Parses the cleaned 'Price Range' string to get the average price.
    Handles single values, ranges with ' - ', and missing values ('-').
    """
    cleaned_price_range = clean_price_range(price_range)

    if cleaned_price_range is None or cleaned_price_range == '-' or cleaned_price_range == '':
        return None

    if ' - ' in cleaned_price_range:
        try:
            low_str, high_str = cleaned_price_range.split(' - ')
            low = float(low_str.strip())
            high = float(high_str.strip())
            return (low + high) / 2
        except ValueError:
            return None
    else:
        try:
            price = float(cleaned_price_range)
            return price
        except ValueError:
            return None

# Or apply to the whole column and inspect the 'Avg. price' column afterwards
ipos_withdrawn['Avg. Price'] = ipos_withdrawn['Clean Price Range'].apply(avg_price_range)


# Display the DataFrame with the new column, including the original Price Range for comparison
ipos_withdrawn[['Clean Price Range', 'Avg. Price']].head(10) # Display first 10 rows


,Clean Price Range,Avg. Price
0,-,NaN
1,3.00 - 4.00,3.50
2,-,NaN
3,-,NaN
4,-,NaN
5,4.00 - 6.00,5.00
6,4.00,4.00
7,8.00 - 10.00,9.00
8,4.00 - 6.00,5.00
9,2.25,2.25


#### Step 4: Convert Shares Offered to numeric, clean missing or invalid values.

In [ ]:
# Step 4: Convert Shares Offered to numeric, clean missing or invalid values.

ipos_withdrawn['Shares Offered'].unique()


array(['-', '2000000', '1400000', '750000', '2775000', '3000000',
       '4444444', '1095000', '2250000', '1500000', '2500000', '3500000',
       '2106250', '3750000', '1333334', '1200000', '500000', '2625000',
       '45000000', '7500000', '2900000', '923077', '2300000', '1000000',
       '1600000', '1250000', '14500000', '1150000', '2651000', '20000000',
       '18000000', '1300000', '21687082', '1270000', '3726709', '1333333',
       '1350000', '4190476', '10000000', '30000000', '25000000',
       '35000000', '15000000', '17500000', '21000000', '26100000',
       '30769231', '16220000'], dtype=object)

In [ ]:
# define the clean column function"

def clean_and_convert(series):
    """
    Cleans non-numeric characters and converts a pandas Series to numeric.
    Assumes the Series contains strings that should be numeric.
    """
    if series.dtype == 'object': # Only apply cleaning to object (string) type columns
        # Clean non-numeric characters (adjust as needed)
        cleaned_series = series.astype(str).str.replace('[$,%]', '', regex=True)
    else:
        cleaned_series = series # If not object type, assume it's already clean or handled

    # Convert to numeric, coercing errors to NaN
    numeric_series = pd.to_numeric(cleaned_series, errors='coerce')

    return numeric_series


In [ ]:
# Clean the 'Shares Offered' column: remove commas and other potential non-numeric characters

columns_to_clean = ['Shares Offered']

for col in columns_to_clean:
    ipos_withdrawn[f'Clean {col}'] = clean_and_convert(ipos_withdrawn[col])

# Display info about the column to see the data type and non-null counts
ipos_withdrawn[f'Clean {col}'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: Clean Shares Offered
Non-Null Count  Dtype  
--------------  -----  
72 non-null     float64
dtypes: float64(1)
memory usage: 932.0 bytes


In [ ]:
ipos_withdrawn[['Shares Offered', 'Clean Shares Offered']].head(10)

,Shares Offered,Clean Shares Offered
0,-,NaN
1,2000000,2000000.0
2,-,NaN
3,-,NaN
4,-,NaN
5,1400000,1400000.0
6,750000,750000.0
7,2775000,2775000.0
8,3000000,3000000.0
9,4444444,4444444.0


In [ ]:
ipos_withdrawn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symbol                100 non-null    object 
 1   Company Name          100 non-null    object 
 2   Price Range           100 non-null    object 
 3   Shares Offered        100 non-null    object 
 4   Clean Company Name    100 non-null    object 
 5   Category              100 non-null    object 
 6   Clean Price Range     100 non-null    object 
 7   Avg. Price            73 non-null     float64
 8   Clean Shares Offered  72 non-null     float64
dtypes: float64(2), object(7)
memory usage: 7.2+ KB


In [ ]:
# Drop null values in 'Avg. Price' and 'Clean Shares Offered' columns

ipos_withdrawn = ipos_withdrawn.dropna(subset=['Avg. Price', 'Clean Shares Offered']).copy()
ipos_withdrawn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, 1 to 99
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symbol                71 non-null     object 
 1   Company Name          71 non-null     object 
 2   Price Range           71 non-null     object 
 3   Shares Offered        71 non-null     object 
 4   Clean Company Name    71 non-null     object 
 5   Category              71 non-null     object 
 6   Clean Price Range     71 non-null     object 
 7   Avg. Price            71 non-null     float64
 8   Clean Shares Offered  71 non-null     float64
dtypes: float64(2), object(7)
memory usage: 5.5+ KB


### Step 5: Create a new column: Withdrawn Value = Shares Offered * Avg Price (71 non-null values).

In [ ]:
# Step 5: Create a new column: Withdrawn Value = Shares Offered * Avg Price (71 non-null values)

ipos_withdrawn['Withdrawn Value'] = ipos_withdrawn['Clean Shares Offered'] * ipos_withdrawn['Avg. Price']

ipos_withdrawn[['Withdrawn Value', 'Clean Shares Offered', 'Avg. Price']].head(10)

,Withdrawn Value,Clean Shares Offered,Avg. Price
1,7000000.0,2000000.0,3.50
5,7000000.0,1400000.0,5.00
6,3000000.0,750000.0,4.00
7,24975000.0,2775000.0,9.00
8,15000000.0,3000000.0,5.00
9,9999999.0,4444444.0,2.25
10,5201250.0,1095000.0,4.75
11,11250000.0,2250000.0,5.00
12,10125000.0,2250000.0,4.50
13,7500000.0,1500000.0,5.00


### Step 6: Group by Company Class and calculate total withdrawn value.


In [ ]:
# Step 6: Group by Company Class and calculate total withdrawn value.

ipos_withdrawn.groupby('Category')['Withdrawn Value'].sum()

,Withdrawn Value
Category,
Acq.Corp,4.021000e+09
Group,3.378750e+07
Holdings,3.030000e+08
Inc,2.257164e+09
Ltd,3.217346e+08
Other,7.679200e+08


## Answer Question 1:
The total withdrawn IPO value (in $ millions) for the company class - **Acquisition Corporations** -  with the highest total withdrawal value is **4021**.


# Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale yfinance OHLCV data download and perform basic financial calculations on IPO stocks.

**Steps:**

1. Using the same approach as in Question 1, download the IPOs in 2024 from:
https://stockanalysis.com/ipos/2024/
Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).
➤ You should have 75 tickers.

2. Use Code Snippet 7 to download daily stock data for those tickers (via yfinance). <br>
Make sure you understand how growth_1d ... growth_365d, and volatility columns are defined.<br>
Define a new column growth_252d representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.

3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

‘stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']‘

4. Filter the DataFrame to keep data only for the trading day:
**‘2025-06-06’**

Compute descriptive statistics (e.g., .describe()) for these columns:

* growth_252d
* Sharpe <br>

You should observe:

* growth_252d is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).
* Median growth_252d is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

### Step 1: Use pandas.read_html() with the URL above to load the IPOs in 2024 table into a DataFrame.

In [ ]:
# Step 1: Use pandas.read_html() with the URL above to load the IPOs in 2024 table into a DataFrame.

from io import StringIO

def get_ipos():
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/2024/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [ ]:
# View info for the data frame

ipos2024 = get_ipos()
ipos2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [ ]:
# View the dataframe

ipos2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"Dec 31, 2024",ONEG,OneConstruction Group Limited,$4.00,$5.05,22.53%
1,"Dec 27, 2024",PHH,"Park Ha Biological Technology Co., Ltd.",$4.00,$25.82,498.75%
2,"Dec 23, 2024",HIT,"Health In Tech, Inc.",$4.00,$0.60,-84.75%
3,"Dec 23, 2024",TDAC,Translational Development Acquisition Corp.,$10.00,$10.26,2.60%
4,"Dec 20, 2024",RANG,Range Capital Acquisition Corp.,$10.00,$10.21,2.00%
...,...,...,...,...,...,...
220,"Jan 18, 2024",CCTG,CCSC Technology International Holdings Limited,$6.00,$1.08,-82.17%
221,"Jan 18, 2024",PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.63,-11.85%
222,"Jan 12, 2024",SYNX,Silynxcom Ltd.,$4.00,$1.90,-56.63%
223,"Jan 11, 2024",SDHC,Smith Douglas Homes Corp.,$21.00,$18.42,-12.10%


In [ ]:
# Filter to drop rows with any missing values and keep only tho IPOs before 1 June 2024

ipos2024['IPO Date'] = pd.to_datetime(ipos2024['IPO Date'], errors='coerce')
ipos2024_to_june = ipos2024[ipos2024['IPO Date'] < pd.Timestamp("2024-06-01")].copy()
ipos2024_to_june

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
148,2024-05-31,NAKA,"Kindly MD, Inc.",-,$13.64,-
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,$17.00,$36.46,114.47%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,$19.56,$18.70,-4.76%
151,2024-05-17,RFAI,RF Acquisition Corp II,$10.00,$10.53,5.30%
152,2024-05-15,JDZG,JIADE Limited,$4.00,$0.29,-92.77%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,$6.00,$1.08,-82.17%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,$16.45,$14.63,-11.85%
222,2024-01-12,SYNX,Silynxcom Ltd.,$4.00,$1.90,-56.63%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,$21.00,$18.42,-12.10%


In [ ]:
# Clean the 'IPO Price', 'Current' and 'Return' columns

columns_to_clean = ['Current', 'IPO Price', 'Return']

for col in columns_to_clean:
    ipos2024_to_june[f'Clean {col}'] = clean_and_convert(ipos2024_to_june[col])

    # Optional: Handle percentage for the 'Return' column
    if col == 'Return':
         ipos2024_to_june[f'Clean {col}'] = ipos2024_to_june[f'Clean {col}'] / 100

    print(f"Cleaned and converted '{col}':")
    ipos2024_to_june[f'Clean {col}'].info()
    print("-" * 50) # Separator for clarity



Cleaned and converted 'Current':
<class 'pandas.core.series.Series'>
Index: 77 entries, 148 to 224
Series name: Clean Current
Non-Null Count  Dtype  
--------------  -----  
77 non-null     float64
dtypes: float64(1)
memory usage: 1.2 KB
--------------------------------------------------
Cleaned and converted 'IPO Price':
<class 'pandas.core.series.Series'>
Index: 77 entries, 148 to 224
Series name: Clean IPO Price
Non-Null Count  Dtype  
--------------  -----  
75 non-null     float64
dtypes: float64(1)
memory usage: 1.2 KB
--------------------------------------------------
Cleaned and converted 'Return':
<class 'pandas.core.series.Series'>
Index: 77 entries, 148 to 224
Series name: Clean Return
Non-Null Count  Dtype  
--------------  -----  
75 non-null     float64
dtypes: float64(1)
memory usage: 1.2 KB
--------------------------------------------------


In [ ]:
# Drop rows with na and show the data frame info
ipos2024_to_june = ipos2024_to_june.dropna(subset=['Clean Current', 'Clean IPO Price', 'Clean Return']).copy()

ipos2024_to_june.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 149 to 224
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   IPO Date         75 non-null     datetime64[ns]
 1   Symbol           75 non-null     object        
 2   Company Name     75 non-null     object        
 3   IPO Price        75 non-null     object        
 4   Current          75 non-null     object        
 5   Return           75 non-null     object        
 6   Clean Current    75 non-null     float64       
 7   Clean IPO Price  75 non-null     float64       
 8   Clean Return     75 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 5.9+ KB


In [ ]:
# Drop rows that are missing values in 'Symbol' or 'IPO Date' columns
ipos2024_to_june = ipos2024_to_june.dropna(subset=['Clean Current', 'Clean IPO Price', 'Clean Return']).copy()

print("\nInfo for DataFrame after dropping missing values in specific columns:")
ipos2024_to_june.info()


Info for DataFrame after dropping missing values in specific columns:
<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 149 to 224
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   IPO Date         75 non-null     datetime64[ns]
 1   Symbol           75 non-null     object        
 2   Company Name     75 non-null     object        
 3   IPO Price        75 non-null     object        
 4   Current          75 non-null     object        
 5   Return           75 non-null     object        
 6   Clean Current    75 non-null     float64       
 7   Clean IPO Price  75 non-null     float64       
 8   Clean Return     75 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 5.9+ KB


### Step 2: Use Code Snippet 7 to download daily stock data for those tickers (via yfinance).

In [ ]:
# Step 2: Code Snippet 7 to download daily stock data for those tickers (via yfinance) - IPOs are for the US stock market

# Create ticker list from symbol column
ticker_list = ipos2024_to_june['Symbol'].unique().tolist()
print(ticker_list)


['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO', 'NNE', 'CCIX', 'VIK', 'ZONE', 'LOAR', 'MRX', 'RBRK', 'NCI', 'MFI', 'YYGH', 'TRSG', 'CDTG', 'CTRI', 'IBTA', 'MTEN', 'TWG', 'ULS', 'PACS', 'MNDR', 'CTNM', 'MAMO', 'ZBAO', 'BOLD', 'MMA', 'UBXG', 'IBAC', 'AUNA', 'BKHA', 'LOBO', 'RDDT', 'ALAB', 'INTJ', 'RYDE', 'LGCL', 'SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'HLXB', 'MGX', 'TBBB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'AVBP', 'BTSG', 'HAO', 'CGON', 'YIBO', 'JL', 'SUGP', 'JVSA', 'KSPI', 'CCTG', 'PSBD', 'SYNX', 'SDHC', 'ROMA']


In [ ]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ticker_list):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [ ]:
# Copy dataframe and convert 'Date' column to a datetime index

stocks_tickers = stocks_df.copy()

# Set the 'Date' column as the DataFrame index
stocks_tickers['Date'] = pd.to_datetime(stocks_tickers['Date'])
stocks_tickers = stocks_tickers.set_index('Date')
stocks_tickers.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23316 entries, 2024-05-23 to 2025-06-20
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Open                           23316 non-null  float64
 1   High                           23316 non-null  float64
 2   Low                            23316 non-null  float64
 3   Close                          23316 non-null  float64
 4   Volume                         23316 non-null  int64  
 5   Dividends                      23316 non-null  float64
 6   Stock Splits                   23316 non-null  float64
 7   Ticker                         23316 non-null  object 
 8   Year                           23316 non-null  int32  
 9   Month                          23316 non-null  int32  
 10  Weekday                        23316 non-null  int32  
 11  growth_1d                      23241 non-null  float64
 12  growth_3d                    

In [ ]:
stocks_tickers

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_252d,growth_365d,growth_future_30d,volatility,is_positive_growth_30d_future
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-23,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.092017,NaN,1
2024-05-24,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,1.079832,NaN,NaN,NaN,NaN,NaN,NaN,0.998054,NaN,0
2024-05-28,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,1.030350,NaN,NaN,NaN,NaN,NaN,NaN,1.001133,NaN,1
2024-05-29,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,0.992825,1.104622,NaN,NaN,NaN,NaN,NaN,0.987828,NaN,0
2024-05-30,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,0.994294,1.017121,NaN,NaN,NaN,NaN,NaN,1.037490,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-13,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,0.923611,0.960289,0.707447,1.934546,3.917526,4.666667,NaN,NaN,10.383358,0
2025-06-16,2.840000,3.000000,2.640000,2.875000,63100,0.0,0.0,ROMA,2025,6,...,1.080827,0.958333,0.805322,1.955782,4.342900,5.424529,NaN,NaN,9.939668,0
2025-06-17,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,0.972174,0.970486,0.755405,1.814935,4.092240,5.008960,NaN,NaN,9.469241,0


### Step 3: Calculate the Sharpe ratio assuming a risk-free rate of 4.5%

In [ ]:
# Step 3: Calculate the Sharpe ratio assuming a risk-free rate of 4.5%

stocks_tickers['Sharpe'] = (stocks_tickers['growth_252d'] - 0.045) / stocks_tickers['volatility']
stocks_tickers['Sharpe']


,Sharpe
Date,
2024-05-23,NaN
2024-05-24,NaN
2024-05-28,NaN
2024-05-29,NaN
2024-05-30,NaN
...,...
2025-06-13,0.445103
2025-06-16,0.541218
2025-06-17,0.524219


### Step 4: Filter the DataFrame to keep data only for the trading day: ‘2025-06-06’

In [ ]:
# Step 4: Filter the DataFrame to keep data only for the trading day: ‘2025-06-06’

specific_date = '2025-06-06'

stocks_6Jun25 = stocks_tickers.loc['2025-06-06'].copy()
stocks_6Jun25 = stocks_6Jun25.dropna(subset=['growth_252d', 'Sharpe']).copy()

print(f"Number of stocks with data on {specific_date}: {len(stocks_6Jun25['growth_252d'])}")


Number of stocks with data on 2025-06-06: 71


In [ ]:
# Compute descriptive statistics
col_stats = stocks_6Jun25[['growth_252d', 'Sharpe']].describe()
col_stats

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152898,0.288285
std,1.406017,0.519028
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668


In [ ]:
# Compare median and mean growth_252d
print(f'You should observe: \n * growth_252d is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).\n * Median growth_252d is approximately 0.75 (indicating a 25% decline), while mean is about 1.15, showing a bias towards high-growth companies pushing the average up.')

median_growth = stocks_6Jun25['growth_252d'].median()
print(f"\nMedian growth_252d for these tickers on {specific_date}: {median_growth:.2f}")

mean_growth = stocks_6Jun25['growth_252d'].mean()
print(f"\nMean growth_252d for these tickers on {specific_date}: {mean_growth:.2f}")


# Get the median Sharpe ratio
median_sharpe = stocks_6Jun25['Sharpe'].median()
print(f"\nMedian Sharpe ratio for these tickers on {specific_date}: {median_sharpe:.2f}")


You should observe: 
 * growth_252d is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
 * Median growth_252d is approximately 0.75 (indicating a 25% decline), while mean is about 1.15, showing a bias towards high-growth companies pushing the average up.

Median growth_252d for these tickers on 2025-06-06: 0.76

Mean growth_252d for these tickers on 2025-06-06: 1.15

Median Sharpe ratio for these tickers on 2025-06-06: 0.08


## Answer Question 2:

The median Sharpe ratio for these **71** stocks is **0.08**.

Note: Positive Sharpe means growth exceeding the risk-free rate of 4.5%.


### [Additional] Do you observe the same top 10 companies when sorting by growth_252d versus sorting by Sharpe?

In [ ]:
# Sort by growth_252d and get top 10 tickers
top10_growth = stocks_6Jun25.sort_values(by='growth_252d', ascending=False).head(10)['Ticker'].tolist()
print("\nTop 10 tickers by growth_252d on 2025-06-06:")
print(top10_growth)

# Sort by Sharpe and get top 10 tickers
top10_sharpe = stocks_6Jun25.sort_values(by='Sharpe', ascending=False).head(10)['Ticker'].tolist()
print("\nTop 10 tickers by Sharpe on 2025-06-06:")
print(top10_sharpe)

# Compare the two lists of tickers
if set(top10_growth) == set(top10_sharpe):
    print("\nThe top 10 companies are the same when sorting by growth_252d and Sharpe.")
else:
    print("\nThe top 10 companies are different when sorting by growth_252d and Sharpe.")

# Find the intersection of the two sets
common_tickers = set(top10_growth).intersection(set(top10_sharpe))

# Get the number and names of matching companies
print("\n Number of mathing tickers: " + str(len(common_tickers)) + "\n Which are: " + str(common_tickers))


Top 10 tickers by growth_252d on 2025-06-06:
['JL', 'ROMA', 'UMAC', 'NNE', 'RBRK', 'AHR', 'AS', 'MRX', 'RDDT', 'MTEN']

Top 10 tickers by Sharpe on 2025-06-06:
['BKHA', 'JVSA', 'LEGT', 'IBAC', 'HLXB', 'MNDR', 'DYCQ', 'INTJ', 'JL', 'TRSG']

The top 10 companies are different when sorting by growth_252d and Sharpe.

 Number of mathing tickers: 1
 Which are: {'JL'}


# Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?** <br>
(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)

---
**Goal:**
>
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.
>
---

**Steps:**

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).<br>
Add **12 new columns:**<br>
future_growth_1m, future_growth_2m, ..., future_growth_12m <br>
(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days) <br>
This logic is similar to historyPrices['growth_future_30d'] from **Code Snippet 7**, but extended to longer timeframes.

2. **Determine the first trading day** (min_date) for each ticker. <br>
This is the earliest date in the data for each stock.

3. **Join the data:** <br>
Perform an **inner join** between the min_date DataFrame and the future growth data on both ticker and date. <br>
➤ You should end up with **75 records** (one per IPO) with all 12 future_growth_... fields populated.

4. **Compute descriptive statistics** for the resulting DataFrame: <br>
Use .describe() or similar to analyze each of the 12 columns:
* future_growth_1m
* future_growth_2m
* ...
* future_growth_12m

5. **Determine the best holding period:** <br>
* Find the number of months (1 to 12) where the average (mean) future growth is maximal.
* This optimal month shows an uplift of >1% compared to all others.
* Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

### Step 1 Start from the existing DataFrame from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

In [ ]:
# Step 1 Start from the existing DataFrame from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

ipos2024_to_june.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 149 to 224
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   IPO Date         75 non-null     datetime64[ns]
 1   Symbol           75 non-null     object        
 2   Company Name     75 non-null     object        
 3   IPO Price        75 non-null     object        
 4   Current          75 non-null     object        
 5   Return           75 non-null     object        
 6   Clean Current    75 non-null     float64       
 7   Clean IPO Price  75 non-null     float64       
 8   Clean Return     75 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 5.9+ KB


In [ ]:
# Add 12 new columns:
# ['future_growth_1m', 'future_growth_2m', ..., 'future_growth_12m']
# (Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ticker_list):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Calculate future growth for 1 to 12 months (assuming 1 month = 21 trading days)
  for month in range(1, 13):
      days = month * 21
      historyPrices[f'future_growth_{month}m'] = historyPrices['Close'].shift(-days) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)


0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [ ]:
# Copy the dataframe and set the 'Date' column as the DataFrame index

stocks_month_growth = stocks_df.copy()

stocks_month_growth['Date'] = pd.to_datetime(stocks_month_growth['Date'])

stocks_month_growth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23316 entries, 0 to 23315
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Open                           23316 non-null  float64       
 1   High                           23316 non-null  float64       
 2   Low                            23316 non-null  float64       
 3   Close                          23316 non-null  float64       
 4   Volume                         23316 non-null  int64         
 5   Dividends                      23316 non-null  float64       
 6   Stock Splits                   23316 non-null  float64       
 7   Ticker                         23316 non-null  object        
 8   Year                           23316 non-null  int32         
 9   Month                          23316 non-null  int32         
 10  Weekday                        23316 non-null  int32         
 11  Date           

### Step 2: Determine the first trading day (min_date) for each ticker.

In [ ]:
# Determine the first trading day (min_date) for each ticker by aggregating the index
# Use the index directly since the 'Date' column was set as the index
min_dates = stocks_df.groupby('Ticker')['Date'].min().reset_index(name='min_date')
min_dates['min_date'] = pd.to_datetime(min_dates['min_date']).copy()

# Display the resulting DataFrame
min_dates

,Ticker,min_date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
70,YIBO,2024-01-25
71,YYGH,2024-04-22
72,ZBAO,2024-04-02
73,ZK,2024-05-10


### Step 3: Join the data

In [ ]:
# Step 3: Join the data
# Perform an inner merge between min_dates and stocks_df
# Join on 'Ticker' and 'Date', where 'Date' in stocks_df matches 'min_date' in min_dates
merged_month_growth = pd.merge(
    min_dates.rename(columns={'min_date': 'Date'}), # Rename 'min_date' to 'Date' for the join
    stocks_month_growth,
    on=['Ticker', 'Date'],
    how='inner'
)

# Display the first few rows and info of the merged DataFrame
print("\nMerged DataFrame head:")
print(merged_month_growth.head())
print("\nMerged DataFrame info:")
merged_month_growth.info()

# Verify the number of records (should be 75, one for each IPO's first trading day)
print(f"\nNumber of records in the merged DataFrame: {len(merged_month_growth)}")




Merged DataFrame head:
  Ticker       Date       Open       High        Low      Close    Volume  \
0    AHR 2024-02-07  12.085785  12.471402  11.878869  12.433781  12732800   
1   ALAB 2024-03-20  52.560001  63.500000  50.610001  62.029999  16843300   
2   ANRO 2024-02-02  22.000000  23.270000  20.000000  20.700001   2386300   
3     AS 2024-02-01  13.400000  13.800000  13.100000  13.400000  18656400   
4   AUNA 2024-03-22   9.510000  10.320000   9.300000   9.600000   9046900   

   Dividends  Stock Splits  Year  ...  future_growth_5m  future_growth_6m  \
0        0.0           0.0  2024  ...          1.214961          1.308118   
1        0.0           0.0  2024  ...          0.631307          0.759794   
2        0.0           0.0  2024  ...          0.563768          0.438164   
3        0.0           0.0  2024  ...          0.935075          0.802239   
4        0.0           0.0  2024  ...          0.782292          0.767708   

   future_growth_7m  future_growth_8m  future_grow

### Step 4: Compute descriptive statistics for the resulting DataFrame.

In [ ]:
# Compute descriptive statistics
# Identify the columns for which to compute descriptive statistics
# These are the 'future_growth_1m' through 'future_growth_12m' columns
growth_columns = [f'future_growth_{month}m' for month in range(1, 13)]

# Select these columns from the merged DataFrame
future_growth_data = merged_month_growth[growth_columns]

# Compute descriptive statistics for the selected columns
future_growth_stats = future_growth_data.describe()

# Display the descriptive statistics
print("Descriptive Statistics for Future Growth Columns (from First Trading Day):")
future_growth_stats

Descriptive Statistics for Future Growth Columns (from First Trading Day):


,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,0.927259,0.940749,0.833988,0.825192,0.803849,0.864267,0.847205,0.833048,0.881849,0.917993,0.883059,0.901667
std,0.346261,0.574267,0.409480,0.401772,0.488226,0.653079,0.712880,0.762356,0.936895,0.911384,0.862902,0.892659
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.659355
75%,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082438,1.049719,1.200799,1.106771,1.136392
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711


In [ ]:
mean_row = future_growth_stats.loc['mean']


# Find the maximum value in this mean_row Series
max_mean_growth_value = mean_row.max()

# Find the index (column name) corresponding to this maximum mean value
optimal_growth_column = mean_row.idxmax()

# Extract the number of months from the column name
optimal_months_str = optimal_growth_column.replace('future_growth_', '').replace('m', '')

try:
    optimal_months = int(optimal_months_str)
    print(f"The optimal number of months to hold for maximum average growth is: {optimal_months} months")
    print(f"The maximum average growth for this period is: {max_mean_growth_value:.4f}")
except ValueError:
    print("Could not determine the month corresponding to the maximum average growth.")

# Optional: Display the mean row for review
print("\nMean values for each holding period:")
mean_row

The optimal number of months to hold for maximum average growth is: 2 months
The maximum average growth for this period is: 0.9407

Mean values for each holding period:


,mean
future_growth_1m,0.927259
future_growth_2m,0.940749
future_growth_3m,0.833988
future_growth_4m,0.825192
future_growth_5m,0.803849
future_growth_6m,0.864267
future_growth_7m,0.847205
future_growth_8m,0.833048
future_growth_9m,0.881849
future_growth_10m,0.917993


## Answer Question 3:

The optimal number of months to hold a newly IPO'd stock for maximum average growth is: 2 months
The maximum average growth for this period is: 0.9407

# Question 4: [Strategy] Simple RSI-Based Trading Strategy

**What is the total profit (in \$thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?**

---
**Goal:**

#### Apply a simple rule-based trading strategy using the **Relative Strength Index (RSI)** technical indicator to identify oversold signals and calculate profits.
---

Steps:

1. **Run the full notebook from Lecture 2 (33 stocks)**

  * Ensure you can generate the merged DataFrame containing:
    * OHLCV data
    * Technical indicators
    * Macro indicators
  * Focus on getting RSI computed using Code Snippets 8 and 9.
  * This process is essential and will help during the capstone project.
2. **Alternative (if tech indicators fail to generate):**
Download precomputed data using this snippet:

In [ ]:
# Step 1 - Generate and import the file containing all the data from the merged data frame

file_id = '1ilqfvkbHh4AO-TARdXzRlieITF_R6yz2'
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
stocks33 = pd.read_parquet("data.parquet", engine="pyarrow")
stocks33


Downloading...
From (original): https://drive.google.com/uc?id=1ilqfvkbHh4AO-TARdXzRlieITF_R6yz2
From (redirected): https://drive.google.com/uc?id=1ilqfvkbHh4AO-TARdXzRlieITF_R6yz2&confirm=t&uuid=ba46184c-ac5f-47f3-8c54-1f1c0c35bdea
To: /content/data.parquet
100%|██████████| 130M/130M [00:01<00:00, 107MB/s]


,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.054277,0.062259,0.054277,0.059598,1.031789e+09,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.059598,0.062791,0.059598,0.061726,3.081600e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.061726,0.063323,0.061726,0.062791,1.331712e+08,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.062791,0.063323,0.060662,0.061194,6.776640e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.061194,0.061726,0.059598,0.060130,4.789440e+07,0.0,0.0,MSFT,1986,1986-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5701,3575.500000,3653.699951,3561.800049,3628.699951,1.322630e+06,0.0,0.0,LT.NS,2025,2025-06-01,...,1.120753,1.194812,0.981504,0.964949,1.011793,1.006653,0.968291,1.034942,1.291087,1.602615
5702,3622.000000,3627.600098,3602.000000,3622.300049,9.306030e+05,0.0,0.0,LT.NS,2025,2025-06-01,...,1.150143,1.269301,1.029075,0.976997,0.979441,0.991739,0.948701,0.982668,1.204330,1.573179
5703,3622.000000,3639.100098,3587.199951,3601.500000,7.433110e+05,0.0,0.0,LT.NS,2025,2025-06-01,...,1.144093,1.234111,1.027324,0.988529,1.002698,0.993665,0.965007,0.993155,1.246131,1.610103
5704,3607.000000,3643.000000,3591.100098,3621.100098,1.580819e+06,0.0,0.0,LT.NS,2025,2025-06-01,...,NaN,NaN,NaN,NaN,0.998102,0.980220,0.988249,0.980272,1.245600,1.611512


In [ ]:
# Sort the data frame by stock(ticker) and date
stocks33 = stocks33.sort_values(by=['Date', 'Ticker'])
stocks33

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
14,0.924636,0.928106,0.914227,0.915962,556800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.915963,0.924637,0.912493,0.922902,321600.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.922901,0.928106,0.922901,0.926371,212800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,0.926371,0.936780,0.917697,0.936780,278400.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,0.936780,0.941985,0.926372,0.931576,268800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230402,787.500000,799.450012,786.099976,796.150024,11367669.0,0.0,0.0,SBIN.NS,2025,2025-06-01,...,1.155526,1.264234,1.018453,0.981082,0.986868,0.987653,0.973783,0.941935,1.232334,1.593579
230403,209.949997,212.600006,209.300003,209.300003,3171340.0,0.0,0.0,SIE.DE,2025,2025-06-01,...,1.155526,1.264234,1.018453,0.981082,0.986868,0.987653,0.973783,0.941935,1.232334,1.593579
230404,3425.000000,3446.300049,3410.899902,3435.699951,2351248.0,0.0,0.0,TCS.NS,2025,2025-06-01,...,1.155526,1.264234,1.018453,0.981082,0.986868,0.987653,0.973783,0.941935,1.232334,1.593579
230405,62.910000,63.189999,62.480000,62.590000,2796700.0,0.0,0.0,TTE,2025,2025-06-01,...,1.155526,1.264234,1.018453,0.981082,0.986868,0.987653,0.973783,0.941935,1.232334,1.593579


In [ ]:
# Drop rows where the Ticker or Date is missing
stocks33 = stocks33.dropna(subset=['Ticker'])
stocks33 = stocks33.dropna(subset=['Date'])
stocks33 = stocks33.dropna(subset=['rsi'])
stocks33 = stocks33.dropna(subset=['growth_future_30d'])
stocks33

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
14,0.924636,0.928106,0.914227,0.915962,556800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,0.915963,0.924637,0.912493,0.922902,321600.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,0.922901,0.928106,0.922901,0.926371,212800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,0.926371,0.936780,0.917697,0.936780,278400.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,0.936780,0.941985,0.926372,0.931576,268800.0,0.0,0.0,LLY,1972,1972-06-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230254,1424.000000,1435.500000,1414.300049,1427.900024,9757276.0,0.0,0.0,RELIANCE.NS,2025,2025-06-01,...,1.144465,1.194753,0.974147,0.963525,1.001529,0.962213,1.016291,1.024643,1.25785,1.589225
230255,2285.000000,2299.000000,2280.000000,2295.000000,64772.0,0.0,0.0,RMS.PA,2025,2025-06-01,...,1.144465,1.194753,0.974147,0.963525,1.001529,0.962213,1.016291,1.024643,1.25785,1.589225
230257,786.250000,796.299988,786.250000,792.349976,11377117.0,0.0,0.0,SBIN.NS,2025,2025-06-01,...,1.144465,1.194753,0.974147,0.963525,1.001529,0.962213,1.016291,1.024643,1.25785,1.589225
230258,212.750000,214.949997,211.850006,214.949997,1221597.0,0.0,0.0,SIE.DE,2025,2025-06-01,...,1.144465,1.194753,0.974147,0.963525,1.001529,0.962213,1.016291,1.024643,1.25785,1.589225


In [ ]:
# Check RSI column
stocks33['rsi'].dtype

dtype('float64')

In [ ]:
# Step 3 and 4
# RSI Strategy Setup: RSI is already available in the dataset as a field.
# The threshold for oversold is defined as RSI < 25
# Filter the dataset by RSI and date:

rsi_threshold = 25
selected_stocks33 = stocks33[
    (stocks33['rsi'] < rsi_threshold) &
    (stocks33['Date'] >= '2000-01-01') &
    (stocks33['Date'] <= '2025-06-01')
]

selected_stocks33

,Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
39088,29.500000,29.580000,28.320000,28.480000,2715000.0,0.0,0.0,BRK-B,2000,2000-02-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39424,18.351364,18.494177,17.137461,17.301695,133888438.0,0.0,0.0,RELIANCE.NS,2000,2000-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39445,17.630163,17.887224,16.423401,16.634048,101621246.0,0.0,0.0,RELIANCE.NS,2000,2000-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39466,16.844696,17.308836,16.005674,16.516228,77665531.0,0.0,0.0,RELIANCE.NS,2000,2000-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39487,16.709024,16.766150,15.323747,15.484410,92389443.0,0.0,0.0,RELIANCE.NS,2000,2000-03-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228911,3285.199980,3285.199980,3210.067973,3219.583252,3512160.0,0.0,0.0,TCS.NS,2025,2025-04-01,...,0.986124,0.910332,0.894344,0.764150,0.989669,0.981027,1.096982,1.013191,0.832490,1.319155
228921,439.402951,444.125586,437.238393,440.977173,3476.0,0.0,0.0,CDI.PA,2025,2025-04-01,...,1.025541,0.950216,0.913442,0.758204,1.004361,1.004169,1.017680,0.999503,0.842386,1.316900
228933,475.759360,481.669414,472.114814,477.926392,1094429.0,0.0,0.0,MC.PA,2025,2025-04-01,...,1.025541,0.950216,0.913442,0.758204,1.004361,1.004169,1.017680,0.999503,0.842386,1.316900
228954,438.025529,447.274040,436.648066,437.238403,7037.0,0.0,0.0,CDI.PA,2025,2025-04-01,...,1.081821,0.978405,0.955568,0.800565,1.010256,1.004180,1.066179,1.026321,0.812695,1.385449


In [ ]:
# Step 5 - Calculate Net Profit Over 25 Years:
# Total number of trades: 1568 | For each trade, you invest $1000 | Use the 30-day forward return (growth_future_30d) to compute net earnings:

net_income = 1000 * (selected_stocks33['growth_future_30d'] - 1).sum().round(3)
net_income

print(f"Total profit (in $thousands): {net_income/1000}")


Total profit (in $thousands): 24.327


### Question 4: Alternative method

In [ ]:
# Step 1 - Generate and import the file containing all the data from the merged data frame

import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=8e4ae483-25e2-4e09-988b-5101c030da86
To: /content/data.parquet
100%|██████████| 130M/130M [00:00<00:00, 165MB/s]


In [ ]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [ ]:
len(selected_df)

1568

In [ ]:
alt_net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum().round(3)
alt_net_income

print(f"Total profit (in $thousands): {alt_net_income/1000}")

Total profit (in $thousands): 24.296


## Answer Question 4:
Total profit (in $thousands): 24


# Question 5. Q5. [Exploratory, Optional] Predicting a Positive-Return IPO

Most of the strategies for investing in IPOs deliver negative average and median returns (and even 75% quantiles).

Question:
How would you change the strategy if you want to increase the profitability?

This is an open-ended brainstorming question — propose ideas for identifying IPOs with positive future returns or building a more effective trading strategy.

I have not considered ways of changing the strategies for investing in IPOs, but by carefully picking IPOs you might be able to improve the returns. For example, focusing on industries/sectors that have historically done well and looking at the underwriters as well as the venture capaitalist and private equity firms involved. Do they have a successful reputation too?